# 4) Markov Chain & Convergence

In [1]:
import numpy as np

### Validate Transition Matrix

**What is this?**
Checks if a matrix is a valid Markov chain transition matrix.

**Requirements for validity:**
1. **Non-negative entries**: All probabilities $P_{ij} \geq 0$
2. **Row stochastic**: Each row sums to 1 (sum of transition probabilities from any state = 1)

**Mathematical Notation:**
- $P$ = Transition probability matrix
- $P_{ij}$ = Probability of transitioning from state $i$ to state $j$
- $i$ = Current state (row index)
- $j$ = Next state (column index)

**Interpretation:**
- $P_{ij}$ = Probability of transitioning from state $i$ to state $j$
- Each row represents a probability distribution over next states

**Example:**
```python
P = [[0.7, 0.3],
     [0.4, 0.6]]  # Valid: rows sum to 1, all non-negative
```

In [2]:
def is_transition_matrix(P, tol=1e-10):
    """
    Check if a matrix is a valid transition matrix.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Matrix to check for transition matrix properties
    tol : float, default=1e-10
        Tolerance for numerical checks (negative values and row sums)
    
    Returns:
    --------
    bool : True if P is a valid transition matrix, False otherwise
    """
    P = np.asarray(P, float)
    if (P < -tol).any():
        return False
    return np.allclose(P.sum(axis=1), 1.0, atol=1e-8)

### Estimate Transition Matrix from Path

**What is this?**
Learn the transition probabilities of a Markov chain from observed state sequences.

**Algorithm:**
1. Count transitions: How many times did we go from state $i$ to state $j$?
2. Normalize each row: $\hat{P}_{ij} = \frac{\text{count}(i \to j)}{\sum_k \text{count}(i \to k)}$

**Variables:**
- $\hat{P}_{ij}$ = Estimated probability of transitioning from state $i$ to state $j$
- $i$ = Current state (source state)
- $j$ = Next state (destination state)
- $k$ = Index for summing over all possible destination states
- count$(i \to j)$ = Number of observed transitions from state $i$ to state $j$

**Example:**
```
Path: [0, 1, 1, 0, 1]
Transitions: 0→1 (2 times), 1→1 (1 time), 1→0 (1 time)
Matrix: P[0,1] = 2/2 = 1.0
        P[1,0] = 1/2 = 0.5
        P[1,1] = 1/2 = 0.5
```

**Use case:** Learning transition probabilities from real data (e.g., customer behavior, weather patterns)

In [3]:
def estimate_transition_matrix(states, n_states=None):
    """
    Estimate transition matrix from observed state sequence.
    
    Parameters:
    -----------
    states : array-like, shape (n_observations,)
        Sequence of observed states (integer state indices)
    n_states : int, optional
        Number of states in the chain. If None, inferred as max(states) + 1
    
    Returns:
    --------
    P_hat : array, shape (n_states, n_states)
        Estimated transition probability matrix where P_hat[i,j] is the 
        estimated probability of transitioning from state i to state j
    """
    s = np.asarray(states, dtype=int)
    if n_states is None:
        n_states = int(s.max()) + 1

    counts = np.zeros((n_states, n_states), dtype=float)
    for a, b in zip(s[:-1], s[1:]):
        counts[a, b] += 1

    row_sums = counts.sum(axis=1, keepdims=True)
    P_hat = np.divide(counts, row_sums, out=np.zeros_like(counts), where=row_sums > 0)
    return P_hat

### Simulate Chain

**What is this?**
Generate a random path/trajectory through a Markov chain.

**Algorithm:**
1. Start at initial state $x_0$
2. At each step, randomly choose next state according to $P[x_t, \cdot]$
3. Repeat for $T$ steps

**Variables:**
- $x_0$ = Initial state (starting state)
- $x_t$ = State at time $t$
- $T$ = Number of transitions to simulate
- $P[x_t, \cdot]$ = Row of transition matrix representing probabilities from state $x_t$ to all other states

**Parameters:**
- `P`: Transition matrix
- `x0`: Starting state
- `T`: Number of transitions to simulate

**Output:** Sequence of states $[x_0, x_1, x_2, \ldots, x_T]$

**Use cases:**
- Simulating random processes (stock prices, weather)
- Monte Carlo estimation of chain properties
- Testing chain convergence

In [4]:
def simulate_chain(P, x0, T, rng=None):
    """
    Simulate a random path through a Markov chain.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    x0 : int
        Initial state (starting state index)
    T : int
        Number of transitions to simulate
    rng : numpy.random.Generator, optional
        Random number generator. If None, creates a new default generator
    
    Returns:
    --------
    path : array, shape (T+1,)
        Sequence of states [x_0, x_1, x_2, ..., x_T] visited during simulation
    """
    rng = np.random.default_rng() if rng is None else rng
    P = np.asarray(P, float)
    x = int(x0)
    path = [x]
    for _ in range(T):
        x = rng.choice(P.shape[0], p=P[x])
        path.append(int(x))
    return np.array(path, dtype=int)

### N-Step Transition Probability

**What is this?**
Computes the probability of being in state $j$ after exactly $n$ steps, starting from state $i$.

**Formula:**
$$P^{(n)}_{ij} = (P^n)_{ij}$$

where $P^n$ is the transition matrix raised to the $n$-th power.

**Variables:**
- $P^{(n)}_{ij}$ = Probability of transitioning from state $i$ to state $j$ in exactly $n$ steps
- $P$ = One-step transition matrix
- $P^n$ = n-step transition matrix (P multiplied by itself n times)
- $i$ = Starting state
- $j$ = Ending state
- $n$ = Number of steps

**Why it works:**
- $P^1 = P$: 1-step transition probabilities
- $P^2 = P \times P$: 2-step transition probabilities  
- $P^n$: n-step transition probabilities (Chapman-Kolmogorov equation)

**Example:**
If you're in state 0, what's the probability of being in state 1 after 5 steps?
```python
prob = n_step_probability(P, start_state=0, end_state=1, n=5)
```

**Use cases:**
- Long-term predictions without computing full stationary distribution
- Finite-horizon forecasting (e.g., "What's my customer state in 6 months?")
- Understanding transient behavior before convergence

In [ ]:
def n_step_probability(P, start_state, end_state, n):
    """
    Calculate probability of being in end_state after n steps from start_state.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    start_state : int
        Starting state index
    end_state : int
        Target state index
    n : int
        Number of steps
        
    Returns:
    --------
    float : Probability of transition from start_state to end_state in n steps
    """
    P = np.asarray(P, float)
    P_n = np.linalg.matrix_power(P, n)
    return P_n[start_state, end_state]


### First Passage Time Probability

**What is this?**
Computes the probability of reaching state $j$ **for the first time** after exactly $n$ steps, starting from state $i$.

**Difference from n-step probability:**
- **N-step probability**: Probability of being in state $j$ after $n$ steps (may have visited before)
- **First passage time**: Probability of reaching state $j$ for the **first time** at step $n$

**Formula:**
$$f^{(n)}_{ij} = P^{(n)}_{ij} - \sum_{k=1}^{n-1} f^{(k)}_{ij} \cdot P^{(n-k)}_{jj}$$

where:
- $f^{(1)}_{ij} = P_{ij}$ (1-step is always first time if $i \neq j$)
- For $i = j$: first return time to the same state

**Variables:**
- $f^{(n)}_{ij}$ = Probability of first passage from state $i$ to state $j$ at exactly step $n$
- $P^{(n)}_{ij}$ = n-step transition probability from $i$ to $j$ (may include multiple visits)
- $k$ = Intermediate step at which first passage could occur
- $P^{(n-k)}_{jj}$ = Probability of being in state $j$ after $n-k$ additional steps from $j$
- $i$ = Starting state
- $j$ = Target state
- $n$ = Exact step at which first passage occurs

**Example:**
If you start in state 0, what's the probability you reach state 2 for the first time at exactly step 5?
```python
prob = first_passage_probability(P, start_state=0, target_state=2, n=5)
```

**Use cases:**
- Time-to-event analysis (e.g., "When will customer first churn?")
- Expected hitting times
- Analyzing recurrence properties

In [ ]:
def first_passage_probability(P, start_state, target_state, n):
    """
    Calculate probability of reaching target_state for the FIRST time 
    at exactly step n, starting from start_state.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    start_state : int
        Starting state index
    target_state : int
        Target state index to reach for first time
    n : int
        Number of steps (must reach at exactly this step)
        
    Returns:
    --------
    float : First passage probability
    """
    P = np.asarray(P, float)
    
    if n == 1:
        # First step: just direct transition probability
        return P[start_state, target_state]
    
    # Compute first passage probabilities recursively
    f = {}  # f[(k, i, j)] = first passage prob from i to j at step k
    
    # Base case: 1-step first passage
    for i in range(P.shape[0]):
        for j in range(P.shape[0]):
            f[(1, i, j)] = P[i, j]
    
    # Recursive computation for steps 2 to n
    for k in range(2, n + 1):
        for i in range(P.shape[0]):
            for j in range(P.shape[0]):
                # P^(k)_{ij} - sum of (first reach j at step m) * (stay around j until step k)
                P_k_ij = n_step_probability(P, i, j, k)
                correction = sum(f[(m, i, j)] * n_step_probability(P, j, j, k - m) 
                               for m in range(1, k))
                f[(k, i, j)] = P_k_ij - correction
    
    return f[(n, start_state, target_state)]


### K-th Passage Time Probability

**What is this?**
Computes the probability of reaching state $j$ **for the k-th time** after exactly $n$ steps, starting from state $i$.

**Generalizes first passage time:**
- $k=1$: First time reaching the state
- $k=2$: Second time reaching the state
- $k=3$: Third time reaching the state, etc.

**Formula (recursive):**
$$f^{(n)}_k(i \to j) = \sum_{m=1}^{n-1} f^{(m)}_{k-1}(i \to j) \cdot f^{(n-m)}_1(j \to j)$$

where:
- $f^{(n)}_1(i \to j)$ is the first passage probability
- $f^{(n)}_k(i \to j)$ builds on previous visits

**Variables:**
- $f^{(n)}_k(i \to j)$ = Probability of k-th passage from state $i$ to state $j$ at exactly step $n$
- $k$ = Which visit/passage (1st, 2nd, 3rd, etc.)
- $m$ = Step at which (k-1)-th passage occurred
- $n-m$ = Remaining steps after (k-1)-th passage
- $f^{(m)}_{k-1}(i \to j)$ = Probability of (k-1)-th passage at step $m$
- $f^{(n-m)}_1(j \to j)$ = Probability of first return to $j$ in remaining $n-m$ steps
- $i$ = Starting state
- $j$ = Target state
- $n$ = Exact step at which k-th passage occurs

**Intuition:**
To reach $j$ for the k-th time at step $n$:
1. Reach $j$ for the (k-1)-th time at some earlier step $m$
2. Then reach $j$ again (first return) in the remaining $n-m$ steps

**Example:**
What's the probability of visiting state 1 for the **second time** at exactly step 10?
```python
prob = kth_passage_probability(P, start_state=0, target_state=1, k=2, n=10)
```

**Use cases:**
- Repeated event analysis (e.g., "When will customer churn for the 2nd time?")
- Multi-visit patterns
- Understanding state recurrence frequency

In [ ]:
def kth_passage_probability(P, start_state, target_state, k, n):
    """
    Calculate probability of reaching target_state for the K-TH time 
    at exactly step n, starting from start_state.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    start_state : int
        Starting state index
    target_state : int
        Target state index
    k : int
        Which visit (1=first time, 2=second time, etc.)
    n : int
        Number of steps (must reach for k-th time at exactly this step)
        
    Returns:
    --------
    float : K-th passage probability
    """
    P = np.asarray(P, float)
    
    if k < 1 or n < k:
        return 0.0  # Impossible to have k visits in less than k steps
    
    # Memoization: f_cache[(visit, step, i, j)] = prob of visit-th passage from i to j at step
    f_cache = {}
    
    def f_passage(visit, step, i, j):
        """Compute visit-th passage probability from i to j at exactly step."""
        if (visit, step, i, j) in f_cache:
            return f_cache[(visit, step, i, j)]
        
        if visit == 1:
            # First passage: use the previous function logic
            result = first_passage_probability(P, i, j, step)
        else:
            # K-th passage: sum over all ways to reach (k-1)-th visit, then first return
            result = 0.0
            for m in range(visit - 1, step):  # Need at least (visit-1) steps for (k-1) visits
                # Reach j for (visit-1)-th time at step m
                prob_prev = f_passage(visit - 1, m, i, j)
                # Then first return from j to j in (step - m) steps
                if step - m > 0:
                    prob_return = first_passage_probability(P, j, j, step - m)
                    result += prob_prev * prob_return
        
        f_cache[(visit, step, i, j)] = result
        return result
    
    return f_passage(k, n, start_state, target_state)


### Irreducibility Check

**What is this?**
A Markov chain is **irreducible** if you can reach any state from any other state (eventually).

**Why it matters:**
- Irreducible chains have a unique stationary distribution
- Non-irreducible chains can get "stuck" in subsets of states

**Algorithm:**
Uses graph reachability: For each state, check if all other states are reachable following the transition graph.

**Example:**
```
Irreducible: [0→1→2→0] (circular)
Not irreducible: [0⇄1] [2⇄3] (two separate groups)
```

**Testing:** The function performs BFS/DFS from each state to verify full connectivity

In [ ]:
def is_irreducible(P):
    """
    Check if a Markov chain is irreducible.
    
    A chain is irreducible if every state is reachable from every other state.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    
    Returns:
    --------
    bool : True if chain is irreducible, False otherwise
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    adj = (P > 0)

    def reachable(start):
        seen = set([start])
        stack = [start]
        while stack:
            u = stack.pop()
            for v in np.where(adj[u])[0]:
                if v not in seen:
                    seen.add(int(v))
                    stack.append(int(v))
        return seen

    for s in range(n):
        if len(reachable(s)) != n:
            return False
    return True

In [17]:
Mat_A = np.array([[0.8,0.2,0,0],
                  [0.6,0.2,0.2,0],
                  [0,0.4,0,0.6],
                  [0,0,0.8,0.2]])

is_irreducible(Mat_A)

True

### Communicating Classes

**What is this?**
States $i$ and $j$ **communicate** if you can go from $i$ to $j$ and from $j$ to $i$ (not necessarily in one step).

**Communicating Class:**
A maximal set of states that all communicate with each other.

**Properties:**
- Communication is an equivalence relation (reflexive, symmetric, transitive)
- Every state belongs to exactly one communicating class
- Irreducible chain = Single communicating class (all states communicate)

**Why it matters:**
- Identifies independent parts of the chain
- States in different classes don't interact
- Each class can have its own stationary distribution

**Example:**
```
States: {0, 1, 2, 3}
Transitions: 0⇄1, 2⇄3 (but no path between {0,1} and {2,3})
Classes: {0, 1} and {2, 3}
```

In [ ]:
def find_communicating_classes(P):
    """
    Find all communicating classes in a Markov chain.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    
    Returns:
    --------
    classes : list of sets
        List of communicating classes (each class is a set of state indices)
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    adj = (P > 0)
    
    def reachable_from(start):
        """Find all states reachable from start."""
        seen = set([start])
        stack = [start]
        while stack:
            u = stack.pop()
            for v in np.where(adj[u])[0]:
                if v not in seen:
                    seen.add(int(v))
                    stack.append(int(v))
        return seen
    
    # Find communicating classes using Union-Find approach
    classes = []
    visited = set()
    
    for i in range(n):
        if i in visited:
            continue
        
        # Find all states reachable from i
        reachable_i = reachable_from(i)
        
        # Find states that can reach i (i.e., i is reachable from them)
        can_reach_i = set()
        for j in range(n):
            if i in reachable_from(j):
                can_reach_i.add(j)
        
        # Communicating class = intersection
        comm_class = reachable_i & can_reach_i
        
        classes.append(comm_class)
        visited.update(comm_class)
    
    return classes

def classify_chain_structure(P):
    """
    Classify the structure of a Markov chain.
    
    Returns information about communicating classes and irreducibility.
    """
    P = np.asarray(P, float)
    classes = find_communicating_classes(P)
    
    is_irreducible_chain = (len(classes) == 1)
    
    return {
        "num_classes": len(classes),
        "classes": classes,
        "is_irreducible": is_irreducible_chain,
        "class_sizes": [len(c) for c in classes]
    }

### Transient and Recurrent States

**What is this?**
Classification of states based on whether the chain will definitely return to them.

**Definitions:**
- **Recurrent state**: Starting from state $i$, the chain returns to $i$ with probability 1
  - Guaranteed to revisit infinitely often
- **Transient state**: Starting from state $i$, there's a chance of never returning
  - Visited only finitely many times (eventually leaves forever)

**Mathematical Test:**
State $i$ is recurrent if and only if:
$$\sum_{n=1}^{\infty} P^n_{ii} = \infty$$

**Key Properties:**
- In a finite irreducible chain, all states are recurrent
- Transient states lead to recurrent states
- If state $i$ is recurrent and communicates with $j$, then $j$ is also recurrent

**Example:**
```
Random walk on integers: All states are recurrent (1D)
Random walk on 2D grid: All states are recurrent
Random walk on 3D grid: All states are transient!
```

**Practical importance:**
- Recurrent states form the "core" of the chain
- Transient states are temporary (system eventually settles in recurrent states)

In [ ]:
def classify_states_transient_recurrent(P, max_steps=1000, tol=1e-10):
    """
    Classify states as transient or recurrent using finite approximation.
    
    For finite chains: A state is recurrent if starting from it, you can return to it.
    More precisely, we check if the sum of return probabilities diverges.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    max_steps : int, default=1000
        Maximum number of steps to check
    tol : float, default=1e-10
        Tolerance for numerical comparisons
    
    Returns:
    --------
    classification : dict
        Dictionary with 'recurrent' and 'transient' state sets
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    recurrent = set()
    transient = set()
    
    # For each state, compute sum of P^n[i,i]
    P_power = P.copy()
    cumulative_return_prob = np.diag(P).copy()
    
    for step in range(2, max_steps + 1):
        P_power = P_power @ P
        cumulative_return_prob += np.diag(P_power)
    
    # In finite chains, we use a heuristic:
    # If sum is very large, state is recurrent
    # More precisely: check if state can reach itself
    for i in range(n):
        # Check if i can return to itself
        can_return = cumulative_return_prob[i] > tol
        
        # For finite chains: state is recurrent if it's in a closed communicating class
        # Simple heuristic: if cumulative return probability is high, it's recurrent
        if cumulative_return_prob[i] > max_steps * 0.01:  # Heuristic threshold
            recurrent.add(i)
        else:
            transient.add(i)
    
    return {
        "recurrent": recurrent,
        "transient": transient,
        "cumulative_return_probs": cumulative_return_prob
    }

def find_absorbing_states(P, tol=1e-10):
    """
    Find absorbing states in a Markov chain.
    
    An absorbing state is a state that, once entered, cannot be left (P[i,i] = 1).
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    tol : float, default=1e-10
        Tolerance for checking P[i,i] = 1
    
    Returns:
    --------
    absorbing_states : set
        Set of absorbing state indices
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    absorbing = set()
    for i in range(n):
        if abs(P[i, i] - 1.0) < tol:
            absorbing.add(i)
    
    return absorbing

### Absorbing States and Absorbing Chains

**What is this?**
An **absorbing state** is a state you can never leave once you enter it.

**Mathematical definition:** State $i$ is absorbing if $P_{ii} = 1$ (and $P_{ij} = 0$ for $j \neq i$)

**Variables:**
- $P_{ii}$ = Probability of staying in state $i$ (equals 1 for absorbing state)
- $P_{ij}$ = Probability of transitioning from state $i$ to state $j$ (equals 0 for absorbing state when $j \neq i$)
- $i$ = State being checked for absorption property
- $j$ = Any other state

**Absorbing Chain:**
A chain with:
1. At least one absorbing state
2. From every state, you can reach some absorbing state

**Standard Form:**
Absorbing chains can be written as:
$$P = \begin{pmatrix} Q & R \\ 0 & I \end{pmatrix}$$

**Standard Form Variables:**
- $Q$ = Transitions between transient states (top-left block)
- $R$ = Transitions from transient to absorbing states (top-right block)
- $0$ = Zero matrix (absorbing states can't transition back to transient)
- $I$ = Identity matrix (absorbing states stay put)

**Fundamental Matrix:** $N = (I - Q)^{-1}$
- $N_{ij}$ = Expected number of times in transient state $j$ starting from transient state $i$
- $I$ = Identity matrix of size (n_transient × n_transient)

**Absorption Probabilities:** $B = NR$
- $B_{ij}$ = Probability of absorbing into absorbing state $j$ starting from transient state $i$

**Examples:**
- Gambler's ruin: Broke ($0) and Rich (target) are absorbing
- Random walk with barriers
- Customer churn models

In [ ]:
def analyze_absorbing_chain(P, absorbing_states=None, tol=1e-10):
    """
    Analyze an absorbing Markov chain.
    
    Computes fundamental matrix and absorption probabilities.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    absorbing_states : set or list, optional
        Indices of absorbing states. If None, automatically detected.
    tol : float, default=1e-10
        Tolerance for numerical operations
    
    Returns:
    --------
    result : dict
        Dictionary containing:
        - 'absorbing_states': Set of absorbing state indices
        - 'transient_states': List of transient state indices
        - 'N': Fundamental matrix (expected time in each transient state)
        - 'B': Absorption probability matrix
        - 'expected_steps': Expected steps to absorption from each transient state
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    # Find absorbing states if not provided
    if absorbing_states is None:
        absorbing_states = find_absorbing_states(P, tol)
    
    absorbing_states = set(absorbing_states)
    transient_states = [i for i in range(n) if i not in absorbing_states]
    
    if len(absorbing_states) == 0:
        return {
            "absorbing_states": set(),
            "transient_states": list(range(n)),
            "N": None,
            "B": None,
            "expected_steps": None,
            "message": "No absorbing states found"
        }
    
    if len(transient_states) == 0:
        return {
            "absorbing_states": absorbing_states,
            "transient_states": [],
            "N": None,
            "B": None,
            "expected_steps": None,
            "message": "All states are absorbing"
        }
    
    # Reorder states: transient first, then absorbing
    absorbing_list = sorted(absorbing_states)
    state_order = transient_states + absorbing_list
    
    # Reorder transition matrix
    P_reordered = P[np.ix_(state_order, state_order)]
    
    # Extract Q (transient to transient) and R (transient to absorbing)
    n_transient = len(transient_states)
    Q = P_reordered[:n_transient, :n_transient]
    R = P_reordered[:n_transient, n_transient:]
    
    # Compute fundamental matrix N = (I - Q)^(-1)
    I = np.eye(n_transient)
    try:
        N = np.linalg.inv(I - Q)
    except np.linalg.LinAlgError:
        return {
            "absorbing_states": absorbing_states,
            "transient_states": transient_states,
            "N": None,
            "B": None,
            "expected_steps": None,
            "message": "Cannot compute fundamental matrix (I-Q is singular)"
        }
    
    # Compute absorption probabilities B = NR
    B = N @ R
    
    # Expected number of steps to absorption
    expected_steps = N @ np.ones(n_transient)
    
    return {
        "absorbing_states": absorbing_states,
        "transient_states": transient_states,
        "N": N,  # Fundamental matrix
        "B": B,  # Absorption probabilities
        "expected_steps": expected_steps,
        "Q": Q,
        "R": R
    }

### Periodicity Check

**What is this?**
A chain is **aperiodic** if states don't have cyclical return patterns. Periodicity matters for convergence.

**Period of a state:** $d = \gcd\{n : P^n_{ii} > 0\}$

**Variables:**
- $d$ = Period of a state (the greatest common divisor)
- $n$ = Number of steps
- $P^n_{ii}$ = Probability of returning to state $i$ after exactly $n$ steps
- $\{n : P^n_{ii} > 0\}$ = Set of all step counts where return is possible
- $\gcd$ = Greatest common divisor function
- $i$ = State being analyzed for periodicity

- Period 1 = Aperiodic (can return at any time)
- Period > 1 = Periodic (returns only at multiples of d)

**Why it matters:**
- Aperiodic + Irreducible → Convergence to unique stationary distribution
- Periodic chains oscillate (don't converge to stationary distribution)

**Making chain aperiodic:**
Add self-loops (e.g., set $P_{ii} > 0$ for some state)

**Example:**
- [0→1→0]: Period 2 (alternates)
- [0→0, 0→1, 1→0]: Period 1 (aperiodic, has self-loop)

In [ ]:
import math

def gcd_list(numbers):
    """
    Compute GCD of a list of numbers.
    
    Parameters:
    -----------
    numbers : list of int
        List of integers to compute GCD for
    
    Returns:
    --------
    int : Greatest common divisor of all numbers in the list
    """
    result = numbers[0]
    for num in numbers[1:]:
        result = math.gcd(result, num)
    return result

def state_period(P, state):
    """
    Compute the period of a specific state in a Markov chain.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    state : int
        State index to check period for
    
    Returns:
    --------
    period : int or float
        Period of the state (1 = aperiodic, >1 = periodic, inf = never returns)
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    max_steps = 100  # Check up to this many steps
    
    # Find all n where P^n[state, state] > 0
    return_times = []
    P_power = P.copy()
    
    for step in range(1, max_steps + 1):
        if P_power[state, state] > 1e-10:
            return_times.append(step)
        P_power = P_power @ P
    
    if len(return_times) == 0:
        return float('inf')  # Never returns
    
    return gcd_list(return_times)

def is_aperiodic(P):
    """
    Check if chain is aperiodic (all states have period 1).
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    
    Returns:
    --------
    bool : True if all states are aperiodic, False otherwise
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    for state in range(n):
        if state_period(P, state) > 1:
            return False
    return True

def all_state_periods(P):
    """
    Compute the period for each state in a Markov chain.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    
    Returns:
    --------
    periods : array, shape (n_states,)
        Array where periods[i] is the period of state i
        (period = 1 means aperiodic, period = inf means never returns)
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    periods = np.zeros(n)
    for state in range(n):
        periods[state] = state_period(P, state)
    
    return periods

### Stationary Distribution

**What is this?**
The stationary (or invariant) distribution is a probability distribution over states that remains unchanged as the Markov chain evolves. Once the chain reaches this distribution, it stays in it forever. This represents the equilibrium behavior of the system.

**Mathematical Definition:** 

A probability distribution $\pi = [\pi_1, \pi_2, \ldots, \pi_n]$ is stationary if:

$$\pi^T P = \pi^T$$

or equivalently:

$$\sum_{j=1}^{n} \pi_j P_{ji} = \pi_i \quad \text{for all } i$$

where:
- $\pi$ is a row vector of probabilities
- $P$ is the transition matrix
- $\pi_i \geq 0$ for all $i$ and $\sum_{i=1}^{n} \pi_i = 1$

**Key Properties:**

1. **Eigenvector Interpretation:** $\pi$ is a left eigenvector of $P$ with eigenvalue 1
   - Equivalently, $\pi$ is a right eigenvector of $P^T$ with eigenvalue 1
   
2. **Uniqueness:** For irreducible, aperiodic chains, the stationary distribution is unique

3. **Convergence:** $\lim_{n \to \infty} P^n = \begin{bmatrix} \pi^T \\ \pi^T \\ \vdots \\ \pi^T \end{bmatrix}$ (each row converges to $\pi^T$)

**How to Find It:**

**Method 1: Eigenvalue Approach**
1. Find eigenvector $v$ of $P^T$ corresponding to eigenvalue $\lambda = 1$
2. Normalize: $\pi = \frac{v}{\sum_i v_i}$ so that $\sum_i \pi_i = 1$

**Method 2: System of Linear Equations**
1. Solve $\pi^T P = \pi^T$ which gives $(n-1)$ independent equations
2. Add constraint $\sum_{i=1}^{n} \pi_i = 1$
3. Solve the resulting system

**Method 3: Power Method**
- Compute $P^n$ for large $n$; any row gives $\pi^T$

**Interpretation:**
- $\pi_i$ = Long-run proportion of time the chain spends in state $i$
- $\pi_i$ = Limiting probability of being in state $i$ after many steps
- After running the chain long enough, state probabilities converge to $\pi$ regardless of initial state
- For irreducible, aperiodic chains, this convergence is guaranteed

**Detailed Example:** 

Weather Model with states $\{$Sunny, Rainy$\}$ and transition matrix:

$$P = \begin{bmatrix} 0.8 & 0.2 \\ 0.4 & 0.6 \end{bmatrix}$$

To find $\pi = [\pi_S, \pi_R]$, solve:
- $0.8\pi_S + 0.4\pi_R = \pi_S$ → $0.2\pi_S = 0.4\pi_R$ → $\pi_S = 2\pi_R$
- $\pi_S + \pi_R = 1$

Solution: $\pi = [0.667, 0.333]$ or $[\frac{2}{3}, \frac{1}{3}]$

**Interpretation:** In the long run, expect 66.7% sunny days and 33.3% rainy days.

In [ ]:
from numpy.linalg import eig


def has_stationary_distribution(P):
    """
    Check if a Markov chain has a unique stationary distribution.
    
    A finite Markov chain has a unique stationary distribution if and only if
    it is irreducible and aperiodic (i.e., ergodic).
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    
    Returns:
    --------
    has_stationary : bool
        True if chain has a unique stationary distribution
    reason : str
        Explanation of why the chain does or doesn't have stationary distribution
    """
    P = np.asarray(P, float)
    
    # Check if it's a valid transition matrix
    if not is_transition_matrix(P):
        return False, "Not a valid transition matrix"
    
    # Check irreducibility
    if not is_irreducible(P):
        return False, "Chain is not irreducible (some states are not reachable from others)"
    
    # Check aperiodicity
    if not is_aperiodic(P):
        return False, "Chain is not aperiodic (has periodic states)"
    
    return True, "Chain is ergodic (irreducible and aperiodic) - has unique stationary distribution"

def stationary_distribution(P):
    """
    Compute the stationary distribution of a Markov chain.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    
    Returns:
    --------
    pi : array, shape (n_states,)
        Stationary distribution where pi[i] is the long-run probability 
        of being in state i. Satisfies: pi^T P = pi^T and sum(pi) = 1
    """
    P = np.asarray(P, dtype=float)
    w, v = eig(P.T)
    k = np.argmin(np.abs(w - 1))
    pi = np.real(v[:, k])
    pi = np.abs(pi)  # Take absolute value instead of maximum with 0
    pi = pi / pi.sum()
    return pi

### Reversibility and Detailed Balance

**What is this?**
A Markov chain is **reversible** if the forward and backward processes are statistically identical. This is formalized through the **detailed balance condition**.

**Detailed Balance Condition:**
A chain with transition matrix $P$ and stationary distribution $\pi$ satisfies detailed balance if:
$$\pi_i P_{ij} = \pi_j P_{ji} \quad \text{for all states } i, j$$

**Variables:**
- $\pi_i$ = Stationary probability of state $i$
- $\pi_j$ = Stationary probability of state $j$
- $P_{ij}$ = Transition probability from state $i$ to state $j$
- $P_{ji}$ = Transition probability from state $j$ to state $i$ (reverse direction)
- $i, j$ = State indices

**Interpretation:**
- $\pi_i P_{ij}$ = Long-run rate of transitions from $i$ to $j$
- $\pi_j P_{ji}$ = Long-run rate of transitions from $j$ to $i$
- Detailed balance means these rates are equal (equilibrium at individual transition level)

**Why it matters:**
- **Checking stationary distribution**: If detailed balance holds, $\pi$ is stationary
- **MCMC algorithms**: Many samplers (Metropolis-Hastings) rely on reversibility
- **Symmetric chains**: If $P = P^T$ (symmetric), the chain is reversible with $\pi = \text{uniform}$

**Variables in Context:**
- $P^T$ = Transpose of transition matrix $P$
- uniform = Uniform distribution where all states have equal probability

**Key Properties:**
- Reversible + Irreducible → Unique stationary distribution
- Not all chains are reversible (e.g., deterministic cycles)
- Reversibility is sufficient but not necessary for stationarity

**Example:**
Random walk on undirected graph is reversible; directed cycle is not.

In [ ]:
def check_detailed_balance(P, pi, tol=1e-8):
    """
    Check if a Markov chain satisfies detailed balance condition.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    pi : array-like
        Stationary distribution (or candidate stationary distribution)
    tol : float, default=1e-8
        Numerical tolerance for equality check
    
    Returns:
    --------
    is_reversible : bool
        True if detailed balance holds
    max_violation : float
        Maximum absolute difference |pi_i * P_ij - pi_j * P_ji|
    """
    P = np.asarray(P, float)
    pi = np.asarray(pi, float)
    n = P.shape[0]
    
    # Check detailed balance: pi[i] * P[i,j] = pi[j] * P[j,i] for all i,j
    max_violation = 0.0
    
    for i in range(n):
        for j in range(n):
            forward_rate = pi[i] * P[i, j]
            backward_rate = pi[j] * P[j, i]
            violation = abs(forward_rate - backward_rate)
            max_violation = max(max_violation, violation)
    
    is_reversible = (max_violation < tol)
    
    return is_reversible, float(max_violation)

def is_reversible_chain(P, pi=None, tol=1e-8):
    """
    Check if a Markov chain is reversible.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    pi : array-like, optional
        Stationary distribution. If None, computes it automatically
    tol : float, default=1e-8
        Numerical tolerance
    
    Returns:
    --------
    reversible : bool
        True if chain is reversible
    message : str
        Explanation
    """
    P = np.asarray(P, float)
    
    # Check if it's a valid transition matrix
    if not is_transition_matrix(P):
        return False, "Not a valid transition matrix"
    
    # Get stationary distribution if not provided
    if pi is None:
        # Check if chain has unique stationary distribution
        has_stat, reason = has_stationary_distribution(P)
        if not has_stat:
            return False, f"Cannot check reversibility: {reason}"
        pi = stationary_distribution(P)
    
    # Check detailed balance
    is_rev, max_viol = check_detailed_balance(P, pi, tol)
    
    if is_rev:
        return True, f"Chain is reversible (max violation: {max_viol:.2e})"
    else:
        return False, f"Chain is not reversible (max violation: {max_viol:.2e})"

def check_symmetry(P, tol=1e-8):
    """
    Check if transition matrix is symmetric (P = P^T).
    Symmetric chains are always reversible with uniform stationary distribution.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    tol : float, default=1e-8
        Numerical tolerance
    
    Returns:
    --------
    is_symmetric : bool
        True if P = P^T
    max_diff : float
        Maximum absolute difference |P_ij - P_ji|
    """
    P = np.asarray(P, float)
    diff = P - P.T
    max_diff = float(np.max(np.abs(diff)))
    is_symmetric = (max_diff < tol)
    
    return is_symmetric, max_diff

### Mean Return Time

**What is this?**
The expected number of steps to return to a state, starting from that state.

**Mathematical definition:** 
For state $i$: $m_i = E[\text{time to return to } i | X_0 = i]$

**Variables:**
- $m_i$ = Mean return time for state $i$ (expected number of steps)
- $E[\cdot]$ = Expected value (average)
- $X_0$ = Initial state at time 0
- $i$ = The state we're returning to

**Relationship to Stationary Distribution:**
For an irreducible, aperiodic chain:
$$\pi_i = \frac{1}{m_i}$$

**Variables in Relationship:**
- $\pi_i$ = Stationary probability of state $i$ (long-run proportion of time in state $i$)
- $m_i$ = Mean return time to state $i$

**Interpretation:**
- $m_i = 10$ means on average, the chain returns to state $i$ every 10 steps
- $\pi_i = 0.1$ means state $i$ is visited 10% of the time
- States with higher stationary probability have shorter return times

**Why it matters:**
- Understanding how "frequently" states are visited
- Quality control: How often does the system return to a desired state?
- Queueing: Average time between customer arrivals

**Example:**
Weather with $\pi_{\text{sunny}} = 0.7$:
- Mean return time to sunny = $1/0.7 \approx 1.43$ days

In [ ]:
def mean_return_times(P, pi=None):
    """
    Compute mean return time for each state.
    
    For an ergodic chain, mean return time m_i = 1 / pi_i.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    pi : array-like, optional
        Stationary distribution. If None, computes it automatically.
    
    Returns:
    --------
    return_times : array
        Mean return time for each state
    """
    P = np.asarray(P, float)
    
    # Get stationary distribution
    if pi is None:
        has_stat, reason = has_stationary_distribution(P)
        if not has_stat:
            raise ValueError(f"Cannot compute return times: {reason}")
        pi = stationary_distribution(P)
    
    pi = np.asarray(pi, float)
    
    # Mean return time = 1 / pi_i (for ergodic chains)
    return_times = np.zeros(len(pi))
    for i in range(len(pi)):
        if pi[i] > 1e-10:
            return_times[i] = 1.0 / pi[i]
        else:
            return_times[i] = np.inf  # State not visited in stationary distribution
    
    return return_times

### Reducing and Reversing Markov Chains

**What is this?**
Techniques for simplifying Markov chains or constructing the time-reversed process.

**1. Canonical Form (Reducing to Standard Form)**

For chains with both transient and recurrent states, reorder the transition matrix into canonical form:
$$P = \begin{pmatrix} Q & R \\ 0 & S \end{pmatrix}$$

Where:
- $Q$: Transitions among transient states
- $R$: Transitions from transient to recurrent
- $S$: Transitions among recurrent states
- $0$: Recurrent states can't return to transient

**2. Lumping States (State Aggregation)**

Combine states into groups if they behave similarly:
- **Lumpable**: If for all states in a group, transition probabilities to other groups are identical
- Reduces state space complexity
- Preserves Markov property if done correctly

**3. Time-Reversed Chain**

The **reversed chain** has transition matrix $\tilde{P}$ where:
$$\tilde{P}_{ij} = \frac{\pi_j P_{ji}}{\pi_i}$$

**Properties:**
- If original chain has stationary distribution $\pi$, reversed chain has the same $\pi$
- Chain is reversible ⟺ Original and reversed chains are identical
- Used in MCMC diagnostics and theoretical analysis

**When to use:**
- **Canonical form**: Analyzing chains with absorbing states
- **Lumping**: Simplifying large state spaces
- **Reversing**: Testing reversibility, understanding detailed balance

In [ ]:
def canonical_form(P, transient_states=None):
    """
    Convert transition matrix to canonical form.
    
    Reorders states so transient states come first, then recurrent states.
    Results in block matrix: [[Q, R], [0, S]]
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    transient_states : list, optional
        List of transient state indices. If None, automatically detected.
    
    Returns:
    --------
    P_canonical : array
        Reordered transition matrix in canonical form
    state_order : array
        New ordering of states
    partition : dict
        Dictionary with 'transient' and 'recurrent' state indices in new order
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    # Detect transient and recurrent states if not provided
    if transient_states is None:
        classification = classify_states_transient_recurrent(P)
        transient_states = list(classification['transient'])
    
    transient_states = list(transient_states)
    recurrent_states = [i for i in range(n) if i not in transient_states]
    
    # Create new state ordering: transient first, then recurrent
    state_order = transient_states + recurrent_states
    
    # Reorder transition matrix
    P_canonical = P[np.ix_(state_order, state_order)]
    
    n_transient = len(transient_states)
    
    return P_canonical, np.array(state_order), {
        'transient': list(range(n_transient)),
        'recurrent': list(range(n_transient, n)),
        'n_transient': n_transient,
        'n_recurrent': len(recurrent_states)
    }

def reverse_chain(P, pi=None):
    """
    Compute the time-reversed Markov chain.
    
    The reversed chain has transition matrix P_tilde where:
    P_tilde[i,j] = (pi[j] * P[j,i]) / pi[i]
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    pi : array-like, optional
        Stationary distribution. If None, computes it automatically.
    
    Returns:
    --------
    P_reversed : array
        Transition matrix of reversed chain
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    # Get stationary distribution
    if pi is None:
        has_stat, reason = has_stationary_distribution(P)
        if not has_stat:
            raise ValueError(f"Cannot compute reversed chain: {reason}")
        pi = stationary_distribution(P)
    
    pi = np.asarray(pi, float)
    
    # Compute reversed transition matrix
    P_reversed = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if pi[i] > 1e-10:
                P_reversed[i, j] = (pi[j] * P[j, i]) / pi[i]
            else:
                P_reversed[i, j] = 0.0
    
    return P_reversed

def test_lumpability(P, partition, tol=1e-8):
    """
    Test if a partition of states is lumpable.
    
    A partition is lumpable if for each group A in the partition,
    all states in A have the same transition probability to each other group B.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    partition : list of lists
        Each element is a list of states that form a group
    tol : float, default=1e-8
        Numerical tolerance
    
    Returns:
    --------
    is_lumpable : bool
        True if partition is lumpable
    lumped_P : array or None
        Transition matrix on lumped state space (if lumpable)
    """
    P = np.asarray(P, float)
    k = len(partition)  # Number of groups
    
    # Check lumpability condition
    lumped_P = np.zeros((k, k))
    
    for a, group_a in enumerate(partition):
        for b, group_b in enumerate(partition):
            # Compute transition probability from group_a to group_b
            # Should be same for all states in group_a
            probs = []
            for i in group_a:
                prob_i_to_b = sum(P[i, j] for j in group_b)
                probs.append(prob_i_to_b)
            
            # Check if all states in group_a have same transition prob to group_b
            if len(probs) > 0:
                avg_prob = np.mean(probs)
                if not all(abs(p - avg_prob) < tol for p in probs):
                    return False, None
                lumped_P[a, b] = avg_prob
    
    return True, lumped_P

### Expected Time to Hit a Target Set

**What is this?**
Calculates the expected number of steps to reach a target state (or set of states) from each starting state.

**Mathematical formulation:**
For state $i$ not in target: $h_i = 1 + \sum_j P_{ij} h_j$
For state $i$ in target: $h_i = 0$

**Variables:**
- $h_i$ = Expected hitting time starting from state $i$ (average number of steps to reach target)
- $P_{ij}$ = Transition probability from state $i$ to state $j$
- $j$ = Index for summing over all possible next states
- $i$ = Current/starting state
- 1 = One step taken before transitioning to next state
- Target = Set of goal states to reach

**How it works:**
Solves a system of linear equations to find expected hitting times.

**Interpretation:**
- $h_i$ = Average number of steps to reach target starting from state $i$
- If target is never reachable from $i$, the value will be infinite

**Example use case:**
- Customer journey: Expected time until purchase
- Game: Expected moves until winning/losing
- Network: Expected hops to reach destination

In [ ]:
def expected_hitting_times(P, target_states):
    """
    Calculate expected number of steps to reach a target set from each state.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    target_states : list or set of int
        Indices of target states to reach
    
    Returns:
    --------
    hitting_times : array, shape (n_states,)
        Expected number of steps to reach target_states from each state.
        For states in target_states, value is 0.
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    target = set(target_states)

    A = np.zeros((n, n), float)
    b = np.zeros(n, float)

    for i in range(n):
        if i in target:
            A[i, i] = 1.0
            b[i] = 0.0
        else:
            A[i, i] = 1.0
            A[i, :] -= P[i, :]
            b[i] = 1.0

    return np.linalg.solve(A, b)

In [23]:
expected_hitting_times(Mat_A, target_states=[2,3])

array([25., 20.,  0.,  0.])

### Convergence By Powering Matrix (K Steps)

**What is this?**
Computes the k-step transition probabilities by raising the transition matrix to the k-th power.

**Formula:** $P^{(k)} = P^k$

**Variables:**
- $P^{(k)}$ = k-step transition matrix
- $P$ = One-step transition matrix
- $k$ = Number of steps (exponent)

**Interpretation:**
- $P^k_{ij}$ = Probability of being in state $j$ after exactly $k$ steps starting from state $i$
- As $k \to \infty$, rows converge to the stationary distribution (for irreducible, aperiodic chains)

**Variables in Interpretation:**
- $i$ = Starting state (row index)
- $j$ = Ending state (column index)
- $\infty$ = Infinity (limit as k approaches infinity)

**Use cases:**
- Predict state distribution after k time steps
- Check convergence to stationary distribution
- Analyze mixing time

**Example:**
```python
P2 = n_step_transition(P, 2)  # 2-step transitions
P100 = n_step_transition(P, 100)  # Should be close to stationary
```

In [ ]:
def n_step_transition(P, k):
    """
    Compute k-step transition probability matrix.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    k : int
        Number of steps
    
    Returns:
    --------
    P_k : array, shape (n_states, n_states)
        k-step transition matrix where P_k[i,j] is the probability 
        of being in state j after exactly k steps starting from state i
    """
    return np.linalg.matrix_power(np.asarray(P, float), k)

### Mixing Time

**What is this?**
The time it takes for a Markov chain to "forget" its initial state and get close to the stationary distribution.

**Mathematical Definition:**
The mixing time $\tau(\epsilon)$ is the smallest time $t$ such that:
$$\max_i \|P^t(i, \cdot) - \pi\|_{TV} \leq \epsilon$$

Where $\|\cdot\|_{TV}$ is the total variation distance.

**Variables:**
- $\tau(\epsilon)$ = Mixing time for tolerance $\epsilon$
- $\epsilon$ = Tolerance level (how close to stationary distribution)
- $t$ = Number of steps (time)
- $i$ = Starting state
- $P^t(i, \cdot)$ = Distribution after $t$ steps starting from state $i$ (row $i$ of $P^t$)
- $\pi$ = Stationary distribution
- $\|\cdot\|_{TV}$ = Total variation distance metric
- $\max_i$ = Maximum over all starting states

**Total Variation Distance:**
$$\|p - q\|_{TV} = \frac{1}{2}\sum_j |p_j - q_j|$$

**Variables in TV Distance:**
- $p, q$ = Two probability distributions
- $p_j, q_j$ = Probability of state $j$ in distributions $p$ and $q$
- $j$ = Index for summing over all states
- $|\cdot|$ = Absolute value

**Interpretation:**
- After $\tau(\epsilon)$ steps, distribution is within $\epsilon$ of stationary
- Common choice: $\epsilon = 0.25$ (within 25% of stationary)
- Smaller mixing time = Faster convergence

**Why it matters:**
- **MCMC**: How many samples to discard as "burn-in"
- **Randomized algorithms**: How long until output is "random enough"
- **Performance**: Fast mixing = Efficient sampling

**Factors affecting mixing time:**
- Chain connectivity (better connected = faster mixing)
- Second largest eigenvalue (closer to 1 = slower mixing)
- Bottlenecks in state space

**Example:**
Well-connected graph: $\tau \approx \log n$
Line graph: $\tau \approx n^2$ (very slow!)

**Variables in Examples:**
- $n$ = Number of states
- $\log n$ = Natural logarithm of $n$

In [ ]:
def total_variation_distance(p, q):
    """
    Compute total variation distance between two probability distributions.
    
    TV(p, q) = 0.5 * sum_i |p_i - q_i|
    
    Parameters:
    -----------
    p, q : array-like
        Probability distributions (should sum to 1)
    
    Returns:
    --------
    tv_dist : float
        Total variation distance (between 0 and 1)
    """
    p = np.asarray(p, float)
    q = np.asarray(q, float)
    
    return 0.5 * np.sum(np.abs(p - q))

def estimate_mixing_time(P, epsilon=0.25, max_steps=10000, pi=None):
    """
    Estimate the mixing time of a Markov chain.
    
    Mixing time is the smallest t such that ||P^t(i,·) - π||_TV ≤ ε for all i.
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    epsilon : float, default=0.25
        Tolerance for total variation distance
    max_steps : int, default=10000
        Maximum number of steps to check
    pi : array-like, optional
        Stationary distribution. If None, computes it automatically.
    
    Returns:
    --------
    mixing_time : int
        Estimated mixing time (number of steps)
    convergence_info : dict
        Information about convergence from each starting state
    """
    P = np.asarray(P, float)
    n = P.shape[0]
    
    # Get stationary distribution
    if pi is None:
        has_stat, reason = has_stationary_distribution(P)
        if not has_stat:
            return None, {"error": f"Cannot compute mixing time: {reason}"}
        pi = stationary_distribution(P)
    
    pi = np.asarray(pi, float)
    
    # For each starting state, find when it gets close to stationary
    convergence_times = np.zeros(n, dtype=int)
    
    for start_state in range(n):
        # Initial distribution (start from state i)
        current_dist = np.zeros(n)
        current_dist[start_state] = 1.0
        
        P_power = np.eye(n)
        
        for t in range(1, max_steps + 1):
            P_power = P_power @ P
            current_dist = P_power[start_state, :]
            
            tv_dist = total_variation_distance(current_dist, pi)
            
            if tv_dist <= epsilon:
                convergence_times[start_state] = t
                break
        else:
            # Didn't converge within max_steps
            convergence_times[start_state] = max_steps
    
    mixing_time = int(np.max(convergence_times))
    
    return mixing_time, {
        "convergence_times": convergence_times,
        "max_time": mixing_time,
        "epsilon": epsilon,
        "all_converged": (mixing_time < max_steps)
    }

def spectral_gap_mixing_bound(P, pi=None):
    """
    Estimate mixing time using spectral gap (second largest eigenvalue).
    
    The spectral gap λ = 1 - |λ_2| where λ_2 is the second largest eigenvalue.
    Mixing time is roughly O(1/λ).
    
    Parameters:
    -----------
    P : array-like
        Transition matrix
    pi : array-like, optional
        Stationary distribution
    
    Returns:
    --------
    spectral_gap : float
        Gap between largest and second largest eigenvalue
    mixing_estimate : float
        Rough estimate of mixing time based on spectral gap
    """
    P = np.asarray(P, float)
    
    # Compute eigenvalues
    eigenvalues = np.linalg.eigvals(P)
    eigenvalues_sorted = np.sort(np.abs(eigenvalues))[::-1]
    
    # Spectral gap = 1 - |second largest eigenvalue|
    if len(eigenvalues_sorted) > 1:
        second_largest = eigenvalues_sorted[1]
        spectral_gap = 1.0 - second_largest
    else:
        spectral_gap = 1.0
    
    # Mixing time estimate: O(log(n) / spectral_gap)
    n = P.shape[0]
    if spectral_gap > 1e-10:
        mixing_estimate = np.log(n) / spectral_gap
    else:
        mixing_estimate = np.inf
    
    return float(spectral_gap), float(mixing_estimate)

### Theoretical Foundations of Markov Chains

This section covers the key mathematical theorems that underpin Markov chain theory.

---

#### Chapman-Kolmogorov Equation

**What is this?**
The fundamental equation describing how multi-step transition probabilities compose.

**Statement:**
$$P^{(n+m)}_{ij} = \sum_{k} P^{(n)}_{ik} P^{(m)}_{kj}$$

In matrix form: $P^{(n+m)} = P^{(n)} \cdot P^{(m)}$

**Variables:**
- $P^{(n+m)}_{ij}$ = Probability of going from state $i$ to state $j$ in $n+m$ steps
- $P^{(n)}_{ik}$ = Probability of going from state $i$ to state $k$ in $n$ steps
- $P^{(m)}_{kj}$ = Probability of going from state $k$ to state $j$ in $m$ steps
- $i$ = Starting state
- $j$ = Ending state
- $k$ = Intermediate state (summation index)
- $n, m$ = Number of steps (positive integers)
- $\sum_{k}$ = Sum over all possible intermediate states

**Interpretation:**
- To go from $i$ to $j$ in $n+m$ steps, you must pass through some intermediate state $k$
- Probability = sum over all possible intermediate states
- This is essentially the Markov property in action

**Consequence:**
- $P^{(n)} = P^n$ (n-fold matrix multiplication)
- Multi-step probabilities can be computed by matrix powers
- Foundational for analyzing long-term behavior

---

#### Limiting Behavior Theorem

**What is this?**
Conditions under which a Markov chain converges to a stationary distribution.

**Statement:**
For a finite, irreducible, and aperiodic Markov chain:
$$\lim_{n \to \infty} P^n_{ij} = \pi_j$$

Where $\pi$ is the unique stationary distribution.

**Variables:**
- $\lim_{n \to \infty}$ = Limit as $n$ approaches infinity
- $P^n_{ij}$ = Probability of being in state $j$ after $n$ steps starting from state $i$
- $\pi_j$ = Stationary probability of state $j$
- $\pi$ = Stationary distribution (vector)
- $i$ = Starting state
- $j$ = Ending state
- $n$ = Number of steps

**What this means:**
- No matter where you start (state $i$), after many steps the probability of being in state $j$ approaches $\pi_j$
- The chain "forgets" its initial state
- All rows of $P^n$ converge to the same vector $\pi$

**Requirements:**
1. **Finite**: Finite number of states
2. **Irreducible**: Can reach any state from any other state
3. **Aperiodic**: No cyclical return patterns

**Visual interpretation:**
```
P^1: Initial transitions
P^2: 2-step transitions  
...
P^100: All rows ≈ [π₀, π₁, π₂, ...]
P^∞: Exact stationary distribution
```

---

#### Ergodic Theorem

**What is this?**
Connects **time averages** (how often you visit states in one long run) with **space averages** (stationary distribution).

**Statement:**
For an irreducible, aperiodic Markov chain with stationary distribution $\pi$:
$$\lim_{n \to \infty} \frac{1}{n} \sum_{k=1}^{n} \mathbb{1}_{X_k = j} = \pi_j \quad \text{almost surely}$$

**Variables:**
- $\lim_{n \to \infty}$ = Limit as $n$ approaches infinity
- $n$ = Total number of steps (time horizon)
- $\frac{1}{n}$ = Normalizing factor (makes it an average)
- $\sum_{k=1}^{n}$ = Sum from time step 1 to $n$
- $k$ = Time step index (summation variable)
- $\mathbb{1}_{X_k = j}$ = Indicator function: equals 1 if $X_k = j$, else 0
- $X_k$ = State at time $k$ (random variable)
- $j$ = Target state
- $\pi_j$ = Stationary probability of state $j$
- "almost surely" = With probability 1 (probabilistic convergence)

**Interpretation:**
- Left side: Fraction of time spent in state $j$ over a long run
- Right side: Stationary probability of state $j$
- They're equal! (with probability 1)

**Practical meaning:**
- Simulate one long chain → frequency of visits estimates $\pi$
- Don't need multiple independent runs
- Justifies using simulation to estimate stationary distribution

**Example:**
Weather chain with $\pi_{\text{sunny}} = 0.7$:
- Run chain for 1000 days
- Count sunny days ≈ 700
- As days → ∞, proportion → 0.7 exactly

**Why it matters:**
- **Monte Carlo methods**: Single long run is enough
- **MCMC sampling**: Frequencies give you the target distribution
- **Real-world validation**: Can estimate $\pi$ from observed data

In [ ]:
def verify_chapman_kolmogorov(P, n, m, tol=1e-10):
    """
    Verify Chapman-Kolmogorov equation: P^(n+m) = P^n * P^m
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    n : int
        First step count
    m : int
        Second step count
    tol : float, default=1e-10
        Numerical tolerance for equality check
    
    Returns:
    --------
    verified : bool
        True if equation holds within tolerance
    max_error : float
        Maximum absolute difference between left and right sides
    """
    P = np.asarray(P, float)
    
    # Compute P^(n+m) directly
    P_n_plus_m = np.linalg.matrix_power(P, n + m)
    
    # Compute P^n * P^m
    P_n = np.linalg.matrix_power(P, n)
    P_m = np.linalg.matrix_power(P, m)
    P_n_times_P_m = P_n @ P_m
    
    # Check if they're equal
    diff = np.abs(P_n_plus_m - P_n_times_P_m)
    max_error = float(np.max(diff))
    verified = (max_error < tol)
    
    return verified, max_error

def demonstrate_limiting_behavior(P, max_steps=100, start_state=0):
    """
    Demonstrate convergence to stationary distribution.
    
    Shows how P^n rows converge to π as n increases.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    max_steps : int, default=100
        Maximum number of steps to compute
    start_state : int, default=0
        Starting state for distribution evolution
    
    Returns:
    --------
    convergence_data : dict
        Dictionary containing:
        - 'steps': Array of step numbers
        - 'distributions': Distribution at each step starting from start_state
        - 'stationary': Stationary distribution π
        - 'distances': Total variation distance from π at each step
    """
    P = np.asarray(P, float)
    n_states = P.shape[0]
    
    # Get stationary distribution
    try:
        pi = stationary_distribution(P)
    except:
        pi = None
    
    # Track distribution starting from start_state
    steps = []
    distributions = []
    distances = []
    
    # Initial distribution (deterministic start)
    current_dist = np.zeros(n_states)
    current_dist[start_state] = 1.0
    
    P_power = np.eye(n_states)
    
    for step in range(max_steps + 1):
        steps.append(step)
        current_dist = P_power[start_state, :]
        distributions.append(current_dist.copy())
        
        if pi is not None:
            tv_dist = total_variation_distance(current_dist, pi)
            distances.append(tv_dist)
        
        if step < max_steps:
            P_power = P_power @ P
    
    return {
        'steps': np.array(steps),
        'distributions': np.array(distributions),
        'stationary': pi,
        'distances': np.array(distances) if pi is not None else None
    }

def simulate_markov_chain(P, start_state, n_steps, rng=None):
    """
    Simulate a Markov chain for n_steps.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    start_state : int
        Initial state index
    n_steps : int
        Number of steps to simulate
    rng : numpy.random.Generator, optional
        Random number generator. If None, creates default generator
    
    Returns:
    --------
    path : array, shape (n_steps+1,)
        Sequence of visited states
    """
    return simulate_chain(P, start_state, n_steps, rng)

def demonstrate_ergodic_theorem(P, start_state=0, n_steps=10000, target_state=None):
    """
    Demonstrate the Ergodic Theorem by simulation.
    
    Shows that time average (frequency of visits) converges to stationary probability.
    
    Parameters:
    -----------
    P : array-like, shape (n_states, n_states)
        Transition probability matrix
    start_state : int, default=0
        Starting state index
    n_steps : int, default=10000
        Number of simulation steps
    target_state : int, optional
        Specific state to track. If None, tracks all states.
    
    Returns:
    --------
    result : dict
        Dictionary containing:
        - 'path': Simulated path (array of states visited)
        - 'time_averages': Frequency of each state over time (n_steps+1 x n_states)
        - 'stationary': Stationary distribution π
        - 'final_frequencies': Final empirical frequencies (should converge to π)
        - 'steps': Array of step numbers
    """
    P = np.asarray(P, float)
    n_states = P.shape[0]
    
    # Simulate path
    path = simulate_markov_chain(P, start_state, n_steps)
    
    # Compute cumulative frequencies
    visit_counts = np.zeros((n_steps + 1, n_states))
    
    for step in range(n_steps + 1):
        if step == 0:
            visit_counts[0, path[0]] = 1
        else:
            visit_counts[step] = visit_counts[step - 1]
            visit_counts[step, path[step]] += 1
    
    # Compute time averages (frequencies)
    time_averages = np.zeros((n_steps + 1, n_states))
    for step in range(1, n_steps + 1):
        time_averages[step] = visit_counts[step] / (step + 1)
    
    # Get stationary distribution
    try:
        pi = stationary_distribution(P)
    except:
        pi = None
    
    return {
        'path': path,
        'time_averages': time_averages,
        'stationary': pi,
        'final_frequencies': time_averages[-1],
        'steps': np.arange(n_steps + 1)
    }